In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lmfit.models import GaussianModel, LinearModel

# Kalibrierung

In [2]:
untergrund = pd.read_csv("Messdaten/kalibrierung/kallibrierung_untergrund.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
am = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Am.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
ba = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Ba.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)
cs = pd.read_csv("Messdaten/kalibrierung/kallibrierung_Cs.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

In [3]:
untergrund.plot(y="rate")
plt.show()

In [4]:
plt.plot(am.rate-untergrund.rate, ' .', label="Messdaten")
plt.legend(loc="best")
plt.show()

In [5]:
plt.plot(ba.rate-untergrund.rate, ' .')
plt.show()

In [6]:
plt.plot(cs.rate-untergrund.rate, ' .')
plt.show()

## Fit der Am Daten

In [7]:
x = np.arange(0,2048)

In [8]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")
gauss_3 = GaussianModel(prefix="gauss_3_")
gauss_4 = GaussianModel(prefix="gauss_4_")
gauss_5 = GaussianModel(prefix="gauss_5_")
gauss_6 = GaussianModel(prefix="gauss_6_")

In [9]:
pars_am_1 = gauss_1.guess(am.rate,x=x)
pars_am_1.update(gauss_2.make_params())
pars_am_1.update(gauss_3.make_params())
pars_am_1.update(gauss_4.make_params())
pars_am_1.update(gauss_0.make_params())
pars_am_2 = gauss_5.guess(am.rate,x=x)
pars_am_2.update(gauss_6.make_params())

In [10]:
# controll the center
pars_am_1['gauss_0_center'].set(50, min=48, max=52)
pars_am_1['gauss_1_center'].set(65, min=63, max=68)
pars_am_1['gauss_2_center'].set(77, min=74, max=80)
pars_am_1['gauss_3_center'].set(98, min=95, max=101)
pars_am_1['gauss_4_center'].set(111, min=108, max=114)
pars_am_2['gauss_5_center'].set(181, min=178, max=184)
pars_am_2['gauss_6_center'].set(213, min=207, max=219)
# controll the amplitude
pars_am_1['gauss_0_amplitude'].set(130*13)#, min=48, max=52)
pars_am_1['gauss_1_amplitude'].set(440*13)#, min=430*2.5, max=440*2.5)
pars_am_1['gauss_2_amplitude'].set(300*13)#, min=290*2.5, max=310*2.5)
pars_am_1['gauss_3_amplitude'].set(310*13)#, min=300*2.5, max=320*2.5)
pars_am_1['gauss_4_amplitude'].set(380*13)#, min=370*2.5, max=390*2.5)
pars_am_2['gauss_5_amplitude'].set(500*13)#, min=490*2.5, max=510*2.5)
pars_am_2['gauss_6_amplitude'].set(2500*13)#, min=2400*2.5, max=2600*2.5)
# controll sigma
pars_am_1['gauss_0_sigma'].set(1, min=0, max=5)
pars_am_1['gauss_1_sigma'].set(5, min=1, max=68)
pars_am_1['gauss_2_sigma'].set(5, min=1, max=80)
pars_am_1['gauss_3_sigma'].set(5, min=1, max=101)
pars_am_1['gauss_4_sigma'].set(5, min=1, max=114)
pars_am_2['gauss_5_sigma'].set(5, min=1, max=184)
pars_am_2['gauss_6_sigma'].set(5, min=1, max=219)

In [11]:
mod_gauss_am_1 = gauss_1 + gauss_2 + gauss_3 + gauss_4 + gauss_0

In [12]:
out_gauss_am_1 = mod_gauss_am_1.fit(am.rate[40:130]-untergrund.rate[40:130],x=x[40:130],params=pars_am_1,weights=1/(am.error[40:130]))

In [51]:
plt.plot(x,out_gauss_am_1.eval(x=np.arange(0,2048)), '-g')
plt.errorbar(x,am.rate-untergrund.rate, yerr=am.error, fmt=' .r')
plt.show()

In [14]:
print(out_gauss_am_1.fit_report())

[[Model]]
    ((((Model(gaussian, prefix='gauss_1_') + Model(gaussian, prefix='gauss_2_')) + Model(gaussian, prefix='gauss_3_')) + Model(gaussian, prefix='gauss_4_')) + Model(gaussian, prefix='gauss_0_'))
[[Fit Statistics]]
    # function evals   = 217
    # data points      = 90
    # variables        = 15
    chi-square         = 86.581
    reduced chi-square = 1.154
[[Variables]]
    gauss_1_sigma:       4.84841741 +/- 0.444386 (9.17%) (init= 5)
    gauss_1_fwhm:        11.4171503 +/- 1.046449 (9.17%)  == '2.3548200*gauss_1_sigma'
    gauss_1_amplitude:   4871.27197 +/- 649.3118 (13.33%) (init= 5720)
    gauss_1_center:      64.5491642 +/- 0.452546 (0.70%) (init= 65)
    gauss_2_sigma:       5.50873396 +/- 0.729345 (13.24%) (init= 5)
    gauss_2_fwhm:        12.9720769 +/- 1.717476 (13.24%)  == '2.3548200*gauss_2_sigma'
    gauss_2_amplitude:   3939.85268 +/- 652.0434 (16.55%) (init= 3900)
    gauss_2_center:      77.2652558 +/- 0.717123 (0.93%) (init= 77)
    gauss_3_sigma:       5

In [15]:
mod_gauss_am_2 = gauss_5 + gauss_6

In [16]:
out_gauss_am_2 = mod_gauss_am_2.fit(am.rate[165:240]-untergrund.rate[165:240],x=x[165:240],params=pars_am_2,weights=1/am.error[165:240])

In [17]:
plt.plot(x,out_gauss_am_2.eval(x=x), '-g')
plt.errorbar(x,am.rate-untergrund.rate, yerr=am.error, fmt=' .r')
plt.show()

In [18]:
print(out_gauss_am_2.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_5_') + Model(gaussian, prefix='gauss_6_'))
[[Fit Statistics]]
    # function evals   = 102
    # data points      = 75
    # variables        = 6
    chi-square         = 115.754
    reduced chi-square = 1.678
[[Variables]]
    gauss_5_sigma:       13.6749687 +/- 0.579408 (4.24%) (init= 5)
    gauss_5_fwhm:        32.2020899 +/- 1.364401 (4.24%)  == '2.3548200*gauss_5_sigma'
    gauss_5_amplitude:   16761.1622 +/- 654.0406 (3.90%) (init= 6500)
    gauss_5_center:      184        +/- 5.647167 (3.07%) (init= 181)
    gauss_6_sigma:       8.56075037 +/- 0.067322 (0.79%) (init= 5)
    gauss_6_fwhm:        20.1590261 +/- 0.158533 (0.79%)  == '2.3548200*gauss_6_sigma'
    gauss_6_amplitude:   52316.5087 +/- 604.2185 (1.15%) (init= 32500)
    gauss_6_center:      213.627553 +/- 0.100067 (0.05%) (init= 213)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_5_sigma, gauss_5_amplitude)  =  0.926 
    C(gauss_5_amplitude, gauss_5_cent

## Fit der Ba Daten

In [19]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")
gauss_3 = GaussianModel(prefix="gauss_3_")
gauss_4 = GaussianModel(prefix="gauss_4_")
gauss_5 = GaussianModel(prefix="gauss_5_")
gauss_6 = GaussianModel(prefix="gauss_6_")

In [20]:
pars_ba_1 = gauss_1.guess(ba.rate,x=x)
pars_ba_2 = gauss_2.guess(ba.rate,x=x)
pars_ba_2.update(gauss_0.make_params())
pars_ba_3 = gauss_3.guess(ba.rate,x=x)
pars_ba_3.update(gauss_4.make_params())
pars_ba_4 = gauss_5.guess(ba.rate,x=x)

In [21]:
# controll the center
pars_ba_1['gauss_1_center'].set(20, min=10, max=25)
pars_ba_2['gauss_2_center'].set(112, min=100, max=120)
pars_ba_2['gauss_0_center'].set(125, min=120, max=130)
pars_ba_3['gauss_3_center'].set(228, min=210, max=235)
pars_ba_3['gauss_4_center'].set(286, min=270, max=300)
pars_ba_4['gauss_5_center'].set(1182, min=1150, max=1200)
# controll the amplitude
pars_ba_1['gauss_1_amplitude'].set(440*13, min=0)#, max=440*2.5)
pars_ba_2['gauss_2_amplitude'].set(300, min=0)#, max=310*2.5)
pars_ba_2['gauss_0_amplitude'].set(125, min=0)#, max=130)
pars_ba_3['gauss_3_amplitude'].set(310*13, min=0)#, max=320*2.5)
pars_ba_3['gauss_4_amplitude'].set(380*13, min=0)#, max=390*2.5)
pars_ba_4['gauss_5_amplitude'].set(500*13, min=0)#, max=510*2.5)
# controll sigma
pars_ba_1['gauss_1_sigma'].set(5, min=1, max=68)
pars_ba_2['gauss_2_sigma'].set(5, min=1, max=80)
pars_ba_3['gauss_3_sigma'].set(5, min=1, max=101)
pars_ba_3['gauss_4_sigma'].set(5, min=1, max=114)
pars_ba_4['gauss_5_sigma'].set(5, min=1, max=184)

In [22]:
mod_ba_1 = gauss_1
out_ba_1 = mod_ba_1.fit(ba.rate[15:25]-untergrund.rate[15:25],params=pars_ba_1,x=x[15:25],weights=1/ba.error[15:25])

In [23]:
plt.plot(x,out_ba_1.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [24]:
print(out_ba_1.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_1_')
[[Fit Statistics]]
    # function evals   = 27
    # data points      = 10
    # variables        = 3
    chi-square         = 7.968
    reduced chi-square = 1.138
[[Variables]]
    gauss_1_sigma:       3.71202384 +/- 0.281089 (7.57%) (init= 5)
    gauss_1_fwhm:        8.74114799 +/- 0.661916 (7.57%)  == '2.3548200*gauss_1_sigma'
    gauss_1_amplitude:   1117.53081 +/- 61.21242 (5.48%) (init= 5720)
    gauss_1_center:      19.1213070 +/- 0.201517 (1.05%) (init= 20)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_1_sigma, gauss_1_amplitude)  =  0.751 
    C(gauss_1_sigma, gauss_1_center)  = -0.209 
    C(gauss_1_amplitude, gauss_1_center)  = -0.204 



In [26]:
mod_ba_2 = gauss_2 + gauss_0
out_ba_2 = mod_ba_2.fit(ba.rate[95:135]-untergrund.rate[95:135],params=pars_ba_2,x=x[95:135],weights=1/ba.error[95:135])

In [27]:
plt.plot(x,out_ba_2.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [28]:
print(out_ba_2.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_2_') + Model(gaussian, prefix='gauss_0_'))
[[Fit Statistics]]
    # function evals   = 134
    # data points      = 40
    # variables        = 6
    chi-square         = 80.259
    reduced chi-square = 2.361
[[Variables]]
    gauss_2_sigma:       6.78656645 +/- 0.105521 (1.55%) (init= 5)
    gauss_2_fwhm:        15.9811424 +/- 0.248484 (1.55%)  == '2.3548200*gauss_2_sigma'
    gauss_2_amplitude:   26863.5431 +/- 382.4157 (1.42%) (init= 300)
    gauss_2_center:      111.053940 +/- 0.126139 (0.11%) (init= 112)
    gauss_0_sigma:       4.59849759 +/- 0.499772 (10.87%) (init= 1)
    gauss_0_fwhm:        10.8286341 +/- 1.176874 (10.87%)  == '2.3548200*gauss_0_sigma'
    gauss_0_amplitude:   2615.75185 +/- 323.1190 (12.35%) (init= 125)
    gauss_0_center:      128.025143 +/- 0.499981 (0.39%) (init= 125)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_0_sigma, gauss_0_amplitude)  =  0.864 
    C(gauss_0_amplitude, gauss_0_cente

In [29]:
mod_ba_3 = gauss_3 + gauss_4
out_ba_3 = mod_ba_3.fit(ba.rate[204:315]-untergrund.rate[204:315],params=pars_ba_3,x=x[204:315],weights=1/ba.error[204:315])

In [30]:
plt.plot(x,out_ba_3.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [31]:
print(out_ba_3.fit_report())

[[Model]]
    (Model(gaussian, prefix='gauss_3_') + Model(gaussian, prefix='gauss_4_'))
[[Fit Statistics]]
    # function evals   = 79
    # data points      = 111
    # variables        = 6
    chi-square         = 186.847
    reduced chi-square = 1.779
[[Variables]]
    gauss_3_sigma:       29.2420708 +/- 3.553440 (12.15%) (init= 5)
    gauss_3_fwhm:        68.8598133 +/- 8.367712 (12.15%)  == '2.3548200*gauss_3_sigma'
    gauss_3_amplitude:   7263.68065 +/- 781.6782 (10.76%) (init= 4030)
    gauss_3_center:      234.999999 +/- 5.363476 (2.28%) (init= 228)
    gauss_4_sigma:       12.5318055 +/- 0.343834 (2.74%) (init= 5)
    gauss_4_fwhm:        29.5101462 +/- 0.809668 (2.74%)  == '2.3548200*gauss_4_sigma'
    gauss_4_amplitude:   11357.4608 +/- 560.8174 (4.94%) (init= 4940)
    gauss_4_center:      284.429428 +/- 0.300680 (0.11%) (init= 286)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_3_sigma, gauss_3_amplitude)  =  0.971 
    C(gauss_3_amplitude, gauss_4_am

In [32]:
mod_ba_4 = gauss_5
out_ba_4 = mod_ba_4.fit(ba.rate[1100:1250],params=pars_ba_4,x=x[1100:1250],weights=1/ba.error[1100:1250])

In [33]:
plt.plot(x,out_ba_4.eval(x=x), '-g')
plt.errorbar(x,ba.rate-untergrund.rate, yerr=ba.error, fmt=' .r')
plt.show()

In [34]:
print(out_ba_4.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_5_')
[[Fit Statistics]]
    # function evals   = 35
    # data points      = 150
    # variables        = 3
    chi-square         = 117.888
    reduced chi-square = 0.802
[[Variables]]
    gauss_5_sigma:       43.3949452 +/- 0.494619 (1.14%) (init= 5)
    gauss_5_fwhm:        102.187285 +/- 1.164740 (1.14%)  == '2.3548200*gauss_5_sigma'
    gauss_5_amplitude:   12789.9828 +/- 117.5778 (0.92%) (init= 6500)
    gauss_5_center:      1180.46864 +/- 0.450395 (0.04%) (init= 1182)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_5_sigma, gauss_5_amplitude)  =  0.430 
    C(gauss_5_sigma, gauss_5_center)  =  0.180 
    C(gauss_5_amplitude, gauss_5_center)  =  0.128 



## Kilibrierung der Cs Daten

In [35]:
gauss_0 = GaussianModel(prefix="gauss_0_")
gauss_1 = GaussianModel(prefix="gauss_1_")
gauss_2 = GaussianModel(prefix="gauss_2_")

In [36]:
pars_cs_1 = gauss_0.guess(cs.rate,x=x)
pars_cs_2 = gauss_1.guess(cs.rate,x=x)
pars_cs_3 = gauss_2.guess(cs.rate,x=x)

In [37]:
# controll the center
pars_cs_1['gauss_0_center'].set(115, min=110, max=120)
pars_cs_2['gauss_1_center'].set(267, min=263, max=270)
pars_cs_3['gauss_2_center'].set(1920, min=1900, max=1940)
# controll the amplitude
pars_cs_1['gauss_0_amplitude'].set(125, min=0)#, max=130)
pars_cs_2['gauss_1_amplitude'].set(440*13, min=0)#, max=440*2.5)
pars_cs_3['gauss_2_amplitude'].set(300, min=0)#, max=310*2.5)
# controll sigma
pars_cs_1['gauss_0_sigma'].set(5, min=1, max=68)
pars_cs_2['gauss_1_sigma'].set(5, min=1, max=68)
pars_cs_3['gauss_2_sigma'].set(5, min=1, max=80)

In [38]:
mod_cs_1 = gauss_0
out_cs_1 = mod_cs_1.fit(cs.rate[100:130],params=pars_cs_1,x=x[100:130],weights=1/cs.error[100:130])

In [39]:
plt.plot(x,out_cs_1.eval(x=x), '-g')
plt.errorbar(x,cs.rate-untergrund.rate, yerr=cs.error, fmt=' .r')
plt.show()

In [40]:
print(out_cs_1.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_0_')
[[Fit Statistics]]
    # function evals   = 37
    # data points      = 30
    # variables        = 3
    chi-square         = 36.299
    reduced chi-square = 1.344
[[Variables]]
    gauss_0_sigma:       8.33786026 +/- 0.166957 (2.00%) (init= 5)
    gauss_0_fwhm:        19.6341601 +/- 0.393154 (2.00%)  == '2.3548200*gauss_0_sigma'
    gauss_0_amplitude:   6336.70331 +/- 103.7750 (1.64%) (init= 125)
    gauss_0_center:      115.366805 +/- 0.153160 (0.13%) (init= 115)
[[Correlations]] (unreported correlations are <  0.100)
    C(gauss_0_sigma, gauss_0_amplitude)  =  0.381 
    C(gauss_0_sigma, gauss_0_center)  =  0.137 



In [41]:
mod_cs_3 = gauss_2
out_cs_3 = mod_cs_3.fit(cs.rate[1860:1980],params=pars_cs_3,x=x[1860:1980],weights=1/cs.error[1860:1980])

In [42]:
plt.plot(x,out_cs_3.eval(x=x), '-g')
plt.errorbar(x,cs.rate-untergrund.rate, yerr=cs.error, fmt=' .r')
plt.show()

In [43]:
print(out_cs_3.fit_report())

[[Model]]
    Model(gaussian, prefix='gauss_2_')
[[Fit Statistics]]
    # function evals   = 31
    # data points      = 120
    # variables        = 3
    chi-square         = 170.233
    reduced chi-square = 1.455
[[Variables]]
    gauss_2_sigma:       23.0352370 +/- 0.152370 (0.66%) (init= 5)
    gauss_2_fwhm:        54.2438368 +/- 0.358805 (0.66%)  == '2.3548200*gauss_2_sigma'
    gauss_2_amplitude:   23839.7666 +/- 187.4390 (0.79%) (init= 300)
    gauss_2_center:      1920.26351 +/- 0.187739 (0.01%) (init= 1920)
[[Correlations]] (unreported correlations are <  0.100)



In [63]:
energien = [14,33,60,31,81,383,32,662]
kanal = [64.5491642,112.029958,213.627553,114.547454,284.659378,1180.46864,115.366805,1920.26351]
kanal_err = [0.452546,0.428068,0.100067,0.241726,0.360883,0.450395,0.153160,0.187739]

In [59]:
plt.errorbar(kanal,energien,yerr=kanal_err, fmt=' .g')
plt.show()

In [64]:
mod_lin = LinearModel()
pars_lin = mod_lin.guess(kanal,x=energien)
out_lin = mod_lin.fit(kanal,params=pars_lin,x=energien,weights=1/(np.array(kanal_err)*10))

In [ ]:
out_lin.plot()
plt.legend(loc="best")
plt.show()

In [66]:
print(out_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 8
    # data points      = 8
    # variables        = 2
    chi-square         = 303.452
    reduced chi-square = 50.575
[[Variables]]
    slope:       2.85594436 +/- 0.023038 (0.81%) (init= 2.889096)
    intercept:   35.9500286 +/- 5.857341 (16.29%) (init= 32.65559)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.549 



# Gradmessung

In [ ]:
grad = pd.read_csv("Messdaten/Grad/Messung_1cmAlu_40Grad_LLDhoch.csv", header=14, delim_whitespace=True, names=['rate','error'], index_col=0)

In [ ]:
grad.plot(y="rate")
plt.show()